In [1]:
!conda info


     active environment : venv
    active env location : /Users/l1x/miniconda3/envs/venv
            shell level : 2
       user config file : /Users/l1x/.condarc
 populated config files : 
          conda version : 4.7.10
    conda-build version : not installed
         python version : 3.7.3.final.0
       virtual packages : 
       base environment : /Users/l1x/miniconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/osx-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /Users/l1x/miniconda3/pkgs
                          /Users/l1x/.conda/pkgs
       envs directories : /Users/l1x/miniconda3/envs
                          /Users/l1x/.conda/envs
               platform : osx-64
             user-agent : conda/4.7.10 requests/2.22.0 CPython/3.7.3 Darwin/18.7.0 OSX/10.14.6
       

In [27]:
import datetime
import pprint
import re

import boto3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns

from pyhive import presto
from scipy import stats
from botocore.exceptions import ClientError

(pd.__version__,sns.__version__, np.__version__)

('0.25.1', '0.9.0', '1.17.0')

In [32]:
## TODO PPRINT

### Generating some example data

In [3]:
sample_size = 10_000_000

f0 = np.random.randint(2, size=sample_size)
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'christopher']
f1 = np.random.choice(aa_milne_arr, size=sample_size, p=[0.5, 0.1, 0.1, 0.3])
f2 = np.random.randint(5_000, size=sample_size)
f3 = np.random.randint(100, size=sample_size)
dl = ['2019-08-23 16:24:55.000', '2019-08-23 15:51:07.000',
       '2019-08-23 15:41:30.000', '2019-08-23 15:36:45.000',
       '2019-08-23 13:54:36.000', '2019-08-23 11:32:04.000',
       '2019-08-23 11:15:20.000', '2019-08-23 11:07:46.000',
       '2019-08-23 10:51:20.000', '2019-08-23 10:51:19.000',
       '2019-08-23 10:33:24.000', '2019-08-23 09:50:18.000',
       '2019-08-23 08:21:51.000', '2019-08-22 11:50:55.000',
       '2019-08-22 11:36:52.000', '2019-08-22 11:31:24.000',
       '2019-08-22 09:56:18.000', '2019-08-21 21:42:50.000',
       '2019-08-21 21:11:08.000', '2019-08-21 21:09:18.000',
       '2019-08-21 21:04:52.000', '2019-08-21 20:57:57.000',
       '2019-08-21 20:27:09.000', '2019-08-21 20:06:50.000',
       '2019-08-21 20:01:00.000', '2019-08-21 19:50:41.000',
       '2019-08-21 17:53:00.000', '2019-08-21 17:38:45.000',
       '2019-08-21 16:37:32.000', '2019-08-15 14:04:18.000',
       '2019-08-15 13:42:21.000', '2019-08-15 13:36:25.000',
       '2019-08-15 13:27:50.000', '2019-08-15 13:22:55.000']
dd = [np.datetime64(d) for d in dl]
f4 = np.random.choice(dd, size=sample_size)
f5 = np.random.randint(500_000, size=sample_size)
sha_arr = [
    '33502f201fb635c6f05664bc2912b6b924756237',
    '441373510f02e9672a1570a47fc3b222f2f95327',
]
f6 = np.random.choice(sha_arr, size=sample_size, p=[0.4, 0.6])
cities_arr = [
    'Seattle',
    'Los Angeles',
    'Portland',
    'San Diego',
    'Phoenix',
    'Dallas',
]
f7 = np.random.choice(cities_arr, size=sample_size, p=[0.1, 0.2, 0.2, 0.2, 0.1, 0.2])
f8 = np.random.random_sample((sample_size,))

In [60]:
df = pd.DataFrame({
    'bin_int': f0, 'milne': f1, 'rint_5_000': f2,
    'rint_100': f3, 'rdates': f4, 'rint_500_000': f5,
    'sha': f6, 'cities': f7, 'rfloat': f8,
})
df.head()

,bin_int,milne,rint_5_000,rint_100,rdates,rint_500_000,sha,cities,rfloat
0,1,pooh,4859,35,2019-08-21 17:53:00,382713,441373510f02e9672a1570a47fc3b222f2f95327,Phoenix,0.697973
1,0,pooh,4081,90,2019-08-23 11:07:46,244716,441373510f02e9672a1570a47fc3b222f2f95327,Seattle,0.459704
2,0,christopher,3412,37,2019-08-23 16:24:55,106630,33502f201fb635c6f05664bc2912b6b924756237,San Diego,0.450522
3,1,pooh,4456,8,2019-08-22 09:56:18,145971,33502f201fb635c6f05664bc2912b6b924756237,San Diego,0.454024
4,1,christopher,3600,14,2019-08-23 11:15:20,497832,441373510f02e9672a1570a47fc3b222f2f95327,San Diego,0.038428


In [5]:
df_sample = df.sample(1000)

In [76]:
file_name = "data.csv"
df.to_csv(file_name, index=False, encoding='utf-8', header=False)

In [8]:
#df_sample.profile_report(style={'full_width':True})

In [19]:
#if you ever store credentials in code you are doing it wrong
#either use instance profile to run inside AWS or use credential profile with local credential files
aws_region = 'eu-west-1'
aws_profile = 'li-istvan'
session = boto3.session.Session(profile_name=aws_profile, region_name=aws_region)
s3_client = session.client('s3', )
s3_resource = session.resource('s3')

In [41]:
aws_s3_bucket = 'li-perf-test'
try:
    s3_client.create_bucket(Bucket=aws_s3_bucket, CreateBucketConfiguration={'LocationConstraint': aws_region})
except ClientError as e:
    if 'BucketAlreadyOwnedByYou' not in str(e):
        raise
    else:
        print("AWS S3 Bucket Already Created: ", aws_s3_bucket)

AWS S3 Bucket Already Created:  li-perf-test


In [42]:
[bucket.name for bucket in s3_resource.buckets.all() if re.match('.*perf.*', bucket.name) is not None]

['li-perf-test']

In [58]:
aws_s3_root_path='data'
aws_s3_object_name = '{aws_s3_root_path}/raw/{file_name}'.format(
    aws_s3_root_path=aws_s3_root_path, 
    file_name=file_name
)
print("File: ", aws_s3_object_name)
try:
    s3_resource.Object(aws_s3_bucket, aws_s3_object_name).load()
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        s3_client.upload_file(file_name, aws_s3_bucket, aws_s3_object_name)
    else:
        raise
except ClientError as e:
    logging.error(e)
else:
    print("File has been uploaded already")

File:  data/raw/data.csv
File has been uploaded already


In [59]:
aws_s3_path = '{aws_s3_root_path}/'.format(aws_s3_root_path=aws_s3_root_path)

paginator = s3_client.get_paginator('list_objects')
filters = { 'Bucket': aws_s3_bucket,
            'Prefix': aws_s3_object_name }
page_iterator = paginator.paginate(**filters)

for page in page_iterator:
    print([p['Key'] for p in page['Contents']])


['data/raw/data.csv']


In [61]:
df.columns

Index(['bin_int', 'milne', 'rint_5_000', 'rint_100', 'rdates', 'rint_500_000',
       'sha', 'cities', 'rfloat'],
      dtype='object')

In [ ]:
'''
CREATE DATABASE IF NOT EXISTS `perf`;
'''

In [77]:
print('''CREATE EXTERNAL TABLE `perf`.`test0`( 
   `bin_int` TINYINT
   , `milne` STRING
   , `rint_5_000` SMALLINT
   , `rint_100` TINYINT
   , `rdates` DATE
   , `rint_500_000` INT
   , `sha` STRING
   , `cities` STRING
   , `rfloat` FLOAT
) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\\n' 
STORED AS TEXTFILE 
LOCATION 's3://{aws_s3_bucket}/{aws_s3_root_path}/raw/';
'''.format(aws_s3_bucket=aws_s3_bucket, aws_s3_root_path=aws_s3_root_path))

CREATE EXTERNAL TABLE `perf`.`test0`( 
   `bin_int` TINYINT
   , `milne` STRING
   , `rint_5_000` SMALLINT
   , `rint_100` TINYINT
   , `rdates` DATE
   , `rint_500_000` INT
   , `sha` STRING
   , `cities` STRING
   , `rfloat` FLOAT
) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
STORED AS TEXTFILE 
LOCATION 's3://li-perf-test/data/raw/';

